In [1]:
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [2]:
class DNN(Model):
    def __init__(self,hidden_dim,output_dim):
        super().__init__()
        self.l1=Dense(hidden_dim,activation='relu')
        self.l2=Dense(hidden_dim,activation='relu')
        self.l3=Dense(hidden_dim,activation='relu')
        self.l4=Dense(output_dim,activation='softmax')
        self.ls=[self.l1,self.l2,self.l3,self.l4]
        
    def call(self,x):
        for layer in self.ls:
            x=layer(x)
        return x

In [3]:
mnist=datasets.mnist
(x_train,t_train),(x_test,t_test)=mnist.load_data()

x_train=(x_train.reshape(-1,784)/255).astype(np.float32)
x_test=(x_test.reshape(-1,784)/255).astype(np.float32)
t_train=np.eye(10)[t_train].astype(np.float32)
t_test=np.eye(10)[t_test].astype(np.float32)

In [4]:
model=DNN(200,10)

In [5]:
criterion=losses.CategoricalCrossentropy()
optimizer=optimizers.SGD(learning_rate=0.1)
train_loss=metrics.Mean()
train_acc=metrics.CategoricalAccuracy()

def compute_loss(t,y):
    return criterion(t,y)

def train_step(x,t):
    with tf.GradientTape() as tape:
        preds=model(x)
        loss=compute_loss(preds,t)
    grads=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    train_loss(loss)
    train_acc(t,preds)
    return loss

epochs=30
batch_size=100
n_batches=x_train.shape[0]//batch_size

for epoch in range(epochs):
    x_,t_=shuffle(x_train,t_train)
    
    for batch in range(n_batches):
        start=batch*batch_size
        end=start+batch_size
        train_step(x_[start:end],t_[start:end])
    
    print('epoch: {}, loss: {:.3f}, acc: {:.3f}'.format(epoch,train_loss.result(),train_acc.result()))


epoch: 0, loss: 4.131, acc: 0.748
epoch: 1, loss: 3.330, acc: 0.796
epoch: 2, loss: 3.005, acc: 0.815
epoch: 3, loss: 2.835, acc: 0.825
epoch: 4, loss: 2.745, acc: 0.831
epoch: 5, loss: 2.687, acc: 0.834
epoch: 6, loss: 2.661, acc: 0.836
epoch: 7, loss: 2.660, acc: 0.836
epoch: 8, loss: 2.701, acc: 0.833
epoch: 9, loss: 2.773, acc: 0.829
epoch: 10, loss: 2.893, acc: 0.821
epoch: 11, loss: 3.345, acc: 0.793
epoch: 12, loss: 4.123, acc: 0.745
epoch: 13, loss: 4.865, acc: 0.699
epoch: 14, loss: 5.508, acc: 0.659
epoch: 15, loss: 6.070, acc: 0.624
epoch: 16, loss: 6.566, acc: 0.593
epoch: 17, loss: 7.008, acc: 0.566
epoch: 18, loss: 7.402, acc: 0.541
epoch: 19, loss: 7.757, acc: 0.519
epoch: 20, loss: 8.079, acc: 0.499
epoch: 21, loss: 8.371, acc: 0.481
epoch: 22, loss: 8.638, acc: 0.464
epoch: 23, loss: 8.882, acc: 0.449
epoch: 24, loss: 9.107, acc: 0.435
epoch: 25, loss: 9.315, acc: 0.422
epoch: 26, loss: 9.507, acc: 0.410
epoch: 27, loss: 9.686, acc: 0.399
epoch: 28, loss: 9.852, acc: 0

In [6]:
test_loss=metrics.Mean()
test_acc=metrics.CategoricalAccuracy()

def test_step(x,t):
    preds=model(x)
    loss=compute_loss(t,preds)
    test_loss(loss)
    test_acc(t,preds)
    return loss

test_step(x_test,t_test)

print('test_loss: {:.3f}, test_acc:{:.3f}'.format(test_loss.result(),test_acc.result()))

test_loss: 14.479, test_acc:0.102
